In [1]:
import pandas as pd

import random
import numpy as np
import joblib

from xgboost import XGBClassifier, plot_importance
from prob_final import get_prob_best, get_prob_add, get_prob_ucb
import lightgbm as lgb

rdseed = 10
random.seed(rdseed)
np.random.seed(rdseed)

num_people = 2000

maxDeltaT = 300  # 120秒为最长采集时差

epsilon = 15

KT_tolerance = 200


def f_epi(delta):
    if delta > epsilon:
        return 0
    else:
        return 1 - delta / epsilon


# 根据一定的特征时间初步关联图码数据-全关联
def get_match_record(imsi, face, deltaSeconds):
    print('getting match record...')
    match_record2 = []
    for i in face.index:
        deviceID = face['DeviceID'][i]
        threshold2 = deltaSeconds[deviceID]
        f = face['FaceLabel'][i]
        ts = face['TimeStamp'][i]
        # 匹配条件：点位相同 且 时间差小于对应deltaTime
        match_imsi2 = imsi[(imsi['DeviceID'] == deviceID) & (abs(imsi['TimeStamp'] - ts) < threshold2)][
            ['Code', 'TimeStamp']]
        if len(match_imsi2) > 0:
            for _, row in match_imsi2.iterrows():
                match_record2.append([f, row['Code'], ts, deviceID, abs(row['TimeStamp'] - ts)])
    df_Match2 = pd.DataFrame(match_record2, columns=['FaceLabel', 'Code', 'TimeStamp', 'DeviceID', 'TimeDelta'])
    print('get_match_record done')
    return df_Match2


# 基于图码预关联的数据，进行特征提取的准备工作
def genFeature(df_Match, df_Face, df_Imsi, deltaSeconds):
    print('generating feature...')
    listDeviceID = list(deltaSeconds.keys())
    listDeviceID.sort()

    # 汇总所有特征

    # 图的总次数
    df_T_temp3 = df_Face.groupby('FaceLabel').count().reset_index()[['FaceLabel', 'DeviceID']]
    df_T_temp3.columns = ['FaceLabel', 'countT']

    df_T_count = df_T_temp3['countT']
    df_T_temp3['countT'] = df_T_count
    
    
    df_T = df_T_temp3

    # 码的总次数
    df_M_temp3 = df_Imsi.groupby('Code').count().reset_index()[['Code', 'DeviceID']]
    df_M_temp3.columns = ['Code', 'countM']

    df_M_count = df_M_temp3['countM']
    df_M_temp3['countM'] = df_M_count

    df_M = df_M_temp3

    # 图码关联的总次数
    df_TM_temp3 = df_Match.groupby(['FaceLabel', 'Code']).count().reset_index()[['FaceLabel', 'Code', 'DeviceID']]

    df_TM_temp3.columns = ['FaceLabel', 'Code', 'countTM']

    df_TM_count = df_TM_temp3['countTM']
    df_TM_temp3['countTM'] = df_TM_count

    df_TM = df_TM_temp3

    # cats & TimeDelta

    df_Cats_temp = df_Match.groupby(['FaceLabel', 'Code']).mean().reset_index()[
        ['FaceLabel', 'Code', 'f_epi', 'TimeDelta']]

    df_Cats_temp.columns = ['FaceLabel', 'Code', 'CATS', 'TimeDelta']
    
    df_Cats_dis = df_Cats_temp['CATS']
    df_Cats_temp['CATS'] = df_Cats_dis

    df_Cats = df_Cats_temp
        
    # KT
    temp = pd.merge_asof(df_Imsi[['Code', 'TimeStamp', 'DeviceID']], df_Face[['FaceLabel', 'TimeStamp', 'DeviceID']],
                         on=["TimeStamp"], by=["DeviceID"], tolerance=KT_tolerance, direction="nearest")
    df_KT = temp.groupby(['Code', 'FaceLabel']).count()[['TimeStamp']]
    df_KT = df_KT.reset_index()
    df_KT.columns = ['Code', 'FaceLabel', 'countKT']
    
    df_KT_count = df_KT['countKT']
    df_KT['countKT'] = df_KT_count

    print('begin merge')

    res = df_TM.merge(df_Cats, on=['FaceLabel', 'Code'], how='left').merge(df_KT, on=['FaceLabel', 'Code'], how='left') \
        .merge(df_T, on='FaceLabel', how='left').merge(df_M, on='Code', how='left')
    
    res['CATS'] = res['CATS'] - 5.5 * pow(res['countTM'], -1/2)
    
    res['TimeDelta'] = res['TimeDelta'] - 5.5 * pow(res['countTM'], -1/2)
    
    res['countKT'].fillna(0, inplace=True)

    print('merge done')

    print('getFeature() done!')

    return res


# 标记图码关联，生成训练数据的label
def label(row):
    p = row['FaceLabel']
    c = row['Code']
    # if(c.endswith(p)):
    if dict_label.get(c) == p:
        return 1
    else:
        return 0


# 根据FaceLabel，标记训练数据
def testflag(row):
    p = row['FaceLabel']
    h = hash(p)
    if h % 100 < 30:  # 30%为测试集
        return 1
    else:
        return 0


# 生成训练的基础数据
def genData(df, multiple):
    df['label'] = df.apply(label, axis=1)
    postive = df[df['label'] == 1]

    negative = df[df['label'] == 0].sample(multiple * len(postive), replace=False, weights="countTM")
    comb = pd.concat([postive, negative])
    # data = comb[comb.columns[2:]] #前两列 FaceLabel Code
    data = comb
    # X = np.array(data[data.columns[:-1]])
    # y = np.array(data['label'])
    X = data[data.columns[:-1]]
    y = data['label']
    return (X, y)


"""
将两张DataFrame进行匹配
匹配规则是TimeStamp最接近者认为属于同一个记录
计算出每个点位的匹配时间差
"""


def getDeltaSeconds(df_Face, df_Imsi):
    # 注意这里的左边是df_Face，右表是df_Imsi
    # 按照最大容忍为120s、最近者匹配的方式进行链接
    # 获得的表应该是在df_Face的基础上
    df_FaceCode = pd.merge_asof(df_Face, df_Imsi, on=["TimeStamp"], by=["FaceLabel", "DeviceID"], tolerance=maxDeltaT,
                                direction="nearest")

    # 创建TimeDelta列表示df_Face中每条记录和其对应最接近的df_Imsi记录之间的时间差。并将nan值转变为0s的时间差
    df_FaceCode["TimeDelta"] = df_FaceCode.apply(
        lambda x: pd.Timedelta(seconds=0) if (pd.isna(x['Time1_x']) or pd.isna(x['Time1_y'])) else abs(
            x['Time1_x'] - x['Time1_y']), axis=1)  # 无法处理NaN
    df_FaceCode['TimeDelta'] = df_FaceCode['TimeDelta'].fillna(pd.Timedelta(seconds=0))

    # 转变为秒为单位
    df_FaceCode['TimeDeltaSeconds'] = df_FaceCode['TimeDelta'].map(lambda x: x.seconds)

    # 对每个人，获取df_FaceCode中在指定点位DeviceID处的每个人的最小匹配时间差
    df_min = df_FaceCode.groupby(["FaceLabel", "DeviceID"])["TimeDeltaSeconds"].min()

    # 对每个点位，分别获取两个指标，分别为平均值+3倍标准差 和 75%位+1.5*(75%位-25%位)
    df_describe = df_min.groupby(["DeviceID"]).describe()
    df_describe['edge1'] = df_describe['mean'] + 3 * df_describe['std']  # 3倍标准差
    df_describe['edge2'] = df_describe['75%'] + 1.5 * (df_describe['75%'] - df_describe['25%'])  # 箱线图四分位确定

    # print(df_describe)
    # 最后返回对是上述两个指标以及最大值中最小的一个
    deltaSeconds = df_describe.apply(lambda x: min(x['edge1'], x['edge2'], x['max']), axis=1)

    return deltaSeconds

In [2]:
path_imsi = 'CCF2021_run_record_c_Train.csv'
path_face = 'CCF2021_run_record_p_Train.csv'

# 进行数据预处理，将中文列名改为英文，并生成时间戳TimeStamp
# 注意这里获得的df_Face里包含了DeviceID列，即点位编号
df_Imsi = pd.read_csv(path_imsi, dtype=str)
df_Imsi.columns = ['DeviceID', 'Lon', 'Lat', 'Time', 'Code']
df_Imsi['Time1'] = pd.to_datetime(df_Imsi['Time'])
df_Imsi['TimeStamp'] = [int(t.timestamp()) for t in df_Imsi['Time1']]
df_Face = pd.read_csv(path_face, dtype=str)
df_Face.columns = ['DeviceID', 'Lon', 'Lat', 'Time', 'FaceLabel']
df_Face['Time1'] = pd.to_datetime(df_Face['Time'])
df_Face['TimeStamp'] = [int(t.timestamp()) for t in df_Face['Time1']]

# 获取人员编号 与 特征码的对应映射字典
path_label = 'CCF2021_run_label_Train.csv'
df_label = pd.read_csv(path_label, dtype=str)
dict_label = {}
for tup in zip(df_label['人员编号'], df_label['特征码']):
    dict_label[tup[1]] = tup[0]

# 按时间进行排序
df_Face = df_Face.sort_values(by="TimeStamp")
df_Imsi = df_Imsi.sort_values(by="TimeStamp")

# 通过映射字典，将对应的人脸识别信息写到识别码的那张DataFrame里
# 经过这步操作后，df_Imsi和df_Face中均含有FaceLabel和DeviceId列
df_Imsi["FaceLabel"] = df_Imsi['Code'].map(lambda x: dict_label.get(x))
deltaSeconds = getDeltaSeconds(df_Face, df_Imsi)

df = get_match_record(df_Imsi, df_Face, deltaSeconds)
df['f_epi'] = df.apply(lambda row: f_epi(row['TimeDelta']), axis=1)

with open('./match_df_win.pic', 'wb') as f:
    joblib.dump(df, f)
    
# with open('./match_df_win.pic', 'rb') as f:
#     df = joblib.load(f)

df

getting match record...
get_match_record done


FaceLabel      Code   TimeStamp DeviceID  TimeDelta     f_epi
0           P0099  CUX4LKr0  1604304642      D00         36  0.000000
1           P0099  CJAslGMR  1604304642      D00         36  0.000000
2           P0099  C8oM1ibW  1604304642      D00         36  0.000000
3           P0099  CKOpg0bP  1604304642      D00         36  0.000000
4           P0099  CuiT3FXg  1604304642      D00         36  0.000000
...           ...       ...         ...      ...        ...       ...
2703735     P1523  C7VRhrzS  1604351909      D02          4  0.733333
2703736     P1523  CT8qXDZf  1604351909      D02          4  0.733333
2703737     P1523  CLdMz6fr  1604351909      D02          4  0.733333
2703738     P1523  COX2fV3y  1604351909      D02          4  0.733333
2703739     P1523  CU8zr7QC  1604351909      D02          2  0.866667

[2703740 rows x 6 columns]

In [3]:
res = genFeature(df, df_Face, df_Imsi, deltaSeconds)
res

generating feature...
begin merge
merge done
getFeature() done!


FaceLabel      Code  countTM      CATS  TimeDelta  countKT  countT  \
0           P0000  C0Dpvdks        1 -5.500000  23.500000      0.0      11   
1           P0000  C0FrLlVu        1 -5.500000  22.500000      0.0      11   
2           P0000  C0G6D2U3        1 -5.500000  14.500000      0.0      11   
3           P0000  C0GqChIs        2 -3.455754   4.610913      0.0      11   
4           P0000  C0HuG5st        1 -5.033333   2.500000      0.0      11   
...           ...       ...      ...       ...        ...      ...     ...   
2142937     P1999  CzjvOroX        1 -5.033333   2.500000      1.0      10   
2142938     P1999  CzkTOQZI        1 -5.500000  12.500000      0.0      10   
2142939     P1999  CzpBxJUK        1 -5.033333   2.500000      0.0      10   
2142940     P1999  CzqJCnLd        1 -5.500000  10.500000      0.0      10   
2142941     P1999  Czy4uRbZ        1 -5.033333   2.500000      1.0      10   

         countM  
0            13  
1            11  
2            13  
3            54  
4            18  
...         ...  
2142937      19  
2142938      18  
2142939      20  
2142940      13  
2142941      19  

[2142942 rows x 8 columns]

In [4]:
# df_Prob = joblib.load(open('./train_prob_10_win.pic', 'rb'))

df_Prob_BEST = get_prob_best(df_Face, df_Imsi)
joblib.dump(df_Prob_BEST, open('./train_prob_best_win.pic', 'wb'))

df_Prob_ADD = get_prob_add(df_Face, df_Imsi)
joblib.dump(df_Prob_BEST, open('./train_prob_add_win.pic', 'wb'))

df_Prob_UCB = get_prob_ucb(df_Face, df_Imsi)
joblib.dump(df_Prob_BEST, open('./train_prob_ucb_win.pic', 'wb'))

Calculating best
 now calculating prob for person NO.1990Calculating ADD
 now calculating prob for person NO.1990Calculating UCB
 now calculating prob for person NO.1990

In [5]:
res = res.merge(df_Prob_BEST, on=['FaceLabel', 'Code'], how='left').merge(df_Prob_ADD, on=['FaceLabel', 'Code'], how='left').merge(df_Prob_UCB, on=['FaceLabel', 'Code'], how='left')
res

FaceLabel      Code  countTM      CATS  TimeDelta  countKT  countT  \
0           P0000  C0Dpvdks        1 -5.500000  23.500000      0.0      11   
1           P0000  C0FrLlVu        1 -5.500000  22.500000      0.0      11   
2           P0000  C0G6D2U3        1 -5.500000  14.500000      0.0      11   
3           P0000  C0GqChIs        2 -3.455754   4.610913      0.0      11   
4           P0000  C0HuG5st        1 -5.033333   2.500000      0.0      11   
...           ...       ...      ...       ...        ...      ...     ...   
2142937     P1999  CzjvOroX        1 -5.033333   2.500000      1.0      10   
2142938     P1999  CzkTOQZI        1 -5.500000  12.500000      0.0      10   
2142939     P1999  CzpBxJUK        1 -5.033333   2.500000      0.0      10   
2142940     P1999  CzqJCnLd        1 -5.500000  10.500000      0.0      10   
2142941     P1999  Czy4uRbZ        1 -5.033333   2.500000      1.0      10   

         countM      Prob  Prob_ADD  Prob_UCB  
0            13 -2.029044 -2.231163  0.447214  
1            11 -1.683198 -1.875135  0.377964  
2            13 -2.041061 -2.234230  0.447214  
3            54 -1.572883 -1.611413  0.377964  
4            18 -2.051210 -2.231698  0.447214  
...         ...       ...       ...       ...  
2142937      19 -2.122527 -2.216723  0.447214  
2142938      18 -3.089695 -2.537931  0.707107  
2142939      20 -2.184027 -2.304185  0.500000  
2142940      13 -2.354929 -2.421405  0.500000  
2142941      19 -2.602048 -2.810010  0.577350  

[2142942 rows x 11 columns]

In [6]:
mid_res = res.copy()  # 保存中间结果

In [7]:
res = mid_res.copy()

In [18]:
res = res[["FaceLabel", "Code", "Prob_ADD", "Prob_UCB"]]

In [19]:
res['label'] = res.apply(label, axis=1)

X = res[res.columns[2:-1]]
y = res['label']

In [20]:
X

Prob_ADD  Prob_UCB
0       -2.231163  0.447214
1       -1.875135  0.377964
2       -2.234230  0.447214
3       -1.611413  0.377964
4       -2.231698  0.447214
...           ...       ...
2142937 -2.216723  0.447214
2142938 -2.537931  0.707107
2142939 -2.304185  0.500000
2142940 -2.421405  0.500000
2142941 -2.810010  0.577350

[2142942 rows x 2 columns]

In [21]:
# XGBoost
model = XGBClassifier(scale_pos_weight=100, learning_rate=0.05, random_state=1000)

print('begin fix')

model.fit(X, y)

print('fix done')

joblib.dump(model, open('./model.pic', 'wb'))

plot_importance(model)

begin fix


D:\Science\Anaconda\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:05:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fix done


<AxesSubplot:title={'center':'Feature importance'}, xlabel='F score', ylabel='Features'>

In [22]:
# XGBoost
resX = res[res.columns[2:-1]]
probability = model.predict_proba(resX)[:, 1]
res['probability'] = pd.Series(probability)

xgb_temp = res.groupby("FaceLabel").apply(lambda t: t[t.probability == t.probability.max()].iloc[0])

xgb_temp["label"] = xgb_temp.apply(label, axis=1)
precision_xgb = len(xgb_temp[xgb_temp['label'] == 1]) / len(xgb_temp)
print("xgboost计算的正确率为：", len(xgb_temp[xgb_temp['label'] == 1]), len(xgb_temp), str(precision_xgb))

xgboost计算的正确率为： 1060 1997 0.5307961942914372


In [ ]:
mid_xgb_temp = xgb_temp.copy()  # 保存中间结果

In [ ]:
iter_times = 0
while True:
    collide_codes = []
    for kc in xgb_temp["Code"]:
        matchs = xgb_temp[xgb_temp['Code'] == kc]
        if len(matchs) > 1:
            collide_codes.append(kc)
    collide_codes = set(collide_codes)
    collide_codes = list(collide_codes)
    if len(collide_codes) < 30 or iter_times > 5:
        break
        
    print("\r", f"当前轮数:{iter_times} 当前碰撞数:{len(collide_codes)}", end='', flush=True)
    iter_times = iter_times+1
    
    for collide_code in collide_codes:

        match_faces = xgb_temp[xgb_temp['Code'] == collide_code]["FaceLabel"].values  # 获取碰撞了的人脸编号
        if len(match_faces) == 1:
            continue
        rel_infos = res[res["FaceLabel"].isin(match_faces)].copy()  # 找到相关的所有记录
        rel_infos.sort_values(by=['probability'], inplace=True, ascending=False)

    #     print(rel_infos)

        distributed_code = []
        face2row = {}
        for _, row in rel_infos.iterrows():
            if row["Code"] in distributed_code or row["FaceLabel"] in face2row:
                continue
            face2row[row["FaceLabel"]] = row
            distributed_code.append(row["Code"])
            if len(face2row) == len(match_faces):
                break

    #     print(face2row)

        if len(face2row) != len(match_faces):
            print("second distribute start")
            for face in match_faces:
                if face not in face2row:
                    face2row[face] = xgb_temp[xgb_temp['FaceLabel'] == face].iloc[0]

        for face in face2row:
            xgb_temp.loc[face] = face2row[face]
  

In [ ]:
precision_xgb = len(xgb_temp[xgb_temp['label'] == 1]) / len(xgb_temp)
print("初步消除碰撞后，xgboost计算的正确率为：", len(xgb_temp[xgb_temp['label'] == 1]), len(xgb_temp), str(precision_xgb))

In [ ]:
temp = xgb_temp[xgb_temp['Code'].isin(collide_codes)].sort_values(by=['Code']).groupby(['Code']).apply(lambda t: t[t.probability == t.probability.max()].iloc[0])
print(f"对{len(temp)}个碰撞Code，指派最高概率的正确数量为{len(temp[temp['label'] == 1])}")

In [ ]:
temp = xgb_temp[xgb_temp['Code'].isin(collide_codes)].sort_values(by=['Code'])
print(f"对{len(temp)}个记录，其正确数量为{len(temp[temp['label'] == 1])}")

In [ ]:
# 重置
xgb_temp = mid_xgb_temp.copy()

In [ ]:
#结束结果探索

In [ ]:
path_imsi = 'CCF2021_run_record_c_EvalB.csv'
path_face = 'CCF2021_run_record_p_EvalB.csv'

df_Imsi = pd.read_csv(path_imsi, dtype=str)
df_Imsi.columns = ['DeviceID', 'Lon', 'Lat', 'Time', 'Code']
df_Imsi['Time1'] = pd.to_datetime(df_Imsi['Time'])
df_Imsi['TimeStamp'] = [int(t.timestamp()) for t in df_Imsi['Time1']]
df_Face = pd.read_csv(path_face, dtype=str)
df_Face.columns = ['DeviceID', 'Lon', 'Lat', 'Time', 'FaceLabel']
df_Face['Time1'] = pd.to_datetime(df_Face['Time'])
df_Face['TimeStamp'] = [int(t.timestamp()) for t in df_Face['Time1']]

df_Face = df_Face.sort_values(by="TimeStamp")
df_Imsi = df_Imsi.sort_values(by="TimeStamp")

df_FaceCode = pd.merge_asof(df_Imsi, df_Face, on=["TimeStamp"], by=["DeviceID"], tolerance=maxDeltaT,
                            direction="nearest")
df_FaceCode["TimeDelta"] = df_FaceCode.apply(
    lambda x: pd.Timedelta(seconds=0) if (pd.isna(x['Time1_x']) or pd.isna(x['Time1_y'])) else abs(
        x['Time1_x'] - x['Time1_y']), axis=1)  # 无法处理NaN
df_FaceCode['TimeDelta'] = df_FaceCode['TimeDelta'].fillna(pd.Timedelta(seconds=0))

df_FaceCode['TimeDeltaSeconds'] = df_FaceCode['TimeDelta'].map(lambda x: x.seconds)

df_min = df_FaceCode.groupby(["Code", "DeviceID"])["TimeDeltaSeconds"].min()

# df_describe = df_min[df_min > 2].groupby(["DeviceID"]).describe()
df_describe = df_min.groupby(["DeviceID"]).describe()
df_describe['edge1'] = df_describe['mean'] + 3 * df_describe['std']  # 3倍标准差

# deltaSeconds = df_describe.apply(lambda x: min(x['edge1'], maxDeltaT), axis=1) #采用3倍标准差方法
deltaSeconds = df_describe.apply(lambda x: min(x['max'], maxDeltaT), axis=1)

# print(df_describe)
# deltaSeconds = {"D00":maxDeltaT, "D01":maxDeltaT, "D02":maxDeltaT, "D03":maxDeltaT, "D04":maxDeltaT, "D05":maxDeltaT}
# print(deltaSeconds)

df = get_match_record(df_Imsi, df_Face, deltaSeconds)
joblib.dump(df, open('./eval_match.pic', 'wb'))

# df = joblib.load(open('./eval_match.pic', 'rb'))

In [ ]:
df

In [ ]:
df['f_epi'] = df.apply(lambda row: f_epi(row['TimeDelta']), axis=1)

res = genFeature(df, df_Face, df_Imsi, deltaSeconds)

df_Prob = get_prob_10(df_Face, df_Imsi)

joblib.dump(df, open('./eval_prob_10.pic', 'wb'))

# df_Prob = joblib.load(open('./eval_prob_10.pic', 'rb'))
res = res.merge(df_Prob, on=['FaceLabel', 'Code'], how='left')
resX = res[res.columns[2:]]

In [ ]:
df_Prob

In [ ]:
# XGBoost
probability = model.predict_proba(resX)[:, 1]
res['probability'] = pd.Series(probability)
res

In [ ]:
res

In [ ]:
xgb_temp = res.groupby("FaceLabel").apply(lambda t: t[t.probability == t.probability.max()].iloc[0])

In [ ]:
iter_times = 0
while True:
    collide_codes = []
    for kc in xgb_temp["Code"]:
        matchs = xgb_temp[xgb_temp['Code'] == kc]
        if len(matchs) > 1:
            collide_codes.append(kc)
    collide_codes = set(collide_codes)
    collide_codes = list(collide_codes)
    if len(collide_codes) < 30 or iter_times > 10:
        break
        
    print("\r", f"当前轮数:{iter_times} 当前碰撞数:{len(collide_codes)}", end='\n')
    iter_times = iter_times+1
    
    for collide_code in collide_codes:

        match_faces = xgb_temp[xgb_temp['Code'] == collide_code]["FaceLabel"].values  # 获取碰撞了的人脸编号
        if len(match_faces) == 1:
            continue
        rel_infos = res[res["FaceLabel"].isin(match_faces)].copy()  # 找到相关的所有记录
        rel_infos.sort_values(by=['probability'], inplace=True, ascending=False)

    #     print(rel_infos)

        distributed_code = []
        face2row = {}
        for _, row in rel_infos.iterrows():
            if row["Code"] in distributed_code or row["FaceLabel"] in face2row:
                continue
            face2row[row["FaceLabel"]] = row
            distributed_code.append(row["Code"])
            if len(face2row) == len(match_faces):
                break

    #     print(face2row)

        if len(face2row) != len(match_faces):
            print("second distribute start")
            for face in match_faces:
                if face not in face2row:
                    face2row[face] = xgb_temp[xgb_temp['FaceLabel'] == face].iloc[0]

        for face in face2row:
            xgb_temp.loc[face] = face2row[face]
  

In [ ]:
import datetime
now = datetime.datetime.now()

path_pred = now.strftime("%H_%M_%S_") + "WIN_CCF2021_run_pred_EvalB_XGBoost.csv"
df_pred = pd.DataFrame(zip(xgb_temp['FaceLabel'], xgb_temp['Code']), columns=['人员编号', '特征码']).sort_values(
    by=['人员编号'])
df_pred.to_csv(path_pred, index=False)